# Mastodon API operations

API operations defined in `mastodon_sim.mastodon_ops`.

In [1]:
%load_ext autoreload
%autoreload 2

## Check environment

Get environment variables with `.env` file (in project root directory)

In [2]:
from mastodon_sim.mastodon_ops import check_env

check_env()

2024-07-06T15:47:06.718487+0000 INFO Successfully loaded .env file.
2024-07-06T15:47:06.719407+0000 INFO API_BASE_URL: https://social-sandbox.com
2024-07-06T15:47:06.719982+0000 INFO EMAIL_PREFIX: austinmw89
2024-07-06T15:47:06.720438+0000 INFO MASTODON_CLIENT_ID: N*****************************************E
2024-07-06T15:47:06.720973+0000 INFO MASTODON_CLIENT_SECRET: b*****************************************s
2024-07-06T15:47:06.721752+0000 INFO USER0003_PASSWORD: f******************************6
2024-07-06T15:47:06.722930+0000 INFO USER0002_PASSWORD: 8******************************b
2024-07-06T15:47:06.723780+0000 INFO USER0004_PASSWORD: 7******************************e
2024-07-06T15:47:06.724351+0000 INFO USER0001_PASSWORD: 9******************************5
2024-07-06T15:47:06.724803+0000 INFO USER0005_PASSWORD: a******************************a


## Get Mastodon client

Create Mastodon client object

In [3]:
from mastodon_sim.mastodon_ops import get_client

mastodon = get_client()

## Login user

Get access token for `user0001`.

In [4]:
from mastodon_sim.mastodon_ops import login

access_token = login("user0001")
print(f"access_token: {access_token}")

access_token: pAvbKpkDFp3kOXZMlwRarowT2H7Ypj4LuyNo6UmX12g


## Update name and bio

Update `user0001`'s display name and bio.

In [5]:
from mastodon_sim.mastodon_ops import update_bio

display_name = "Bob"
bio = "Hello, I am user0001!"

update_bio(login_user="user0001", display_name=display_name, bio=bio)

2024-07-06T15:47:07.950649+0000 INFO user0001 successfully updated the display name and bio.


## Read name and bio

Have one user read another user's bio. Defaults to "Not provided" if the user has not set either.

In [6]:
from mastodon_sim.mastodon_ops import read_bio

display_name, bio = read_bio(login_user="user0002", target_user="user0001")

display_name, bio = read_bio(login_user="user0001", target_user="user0002")

2024-07-06T15:47:09.153100+0000 INFO User: user0001
Display Name: Bob
Bio: <p>Hello, I am user0001!</p>
2024-07-06T15:47:09.564339+0000 INFO User: user0002
Display Name: Not provided
Bio: Not provided


## Follow user

Make `user0001` follow `user0002` and vice versa.

In [7]:
from mastodon_sim.mastodon_ops import follow

follow(login_user="user0001", follow_user="user0002")

follow(login_user="user0002", follow_user="user0001")

2024-07-06T15:47:10.146948+0000 INFO user0001 is now following user0002.
2024-07-06T15:47:10.634085+0000 INFO user0002 is now following user0001.


## Unfollow user

Make `user0002` unfollow `user0001`.

In [8]:
from mastodon_sim.mastodon_ops import unfollow

unfollow(login_user="user0002", unfollow_user="user0001")

2024-07-06T15:47:11.098197+0000 INFO user0002 has unfollowed user0001.


## Toot (post a status)

Make `user0001` toot a `status`.

Note that "toot" is analogous to "tweet".

In [9]:
from mastodon_sim.mastodon_ops import toot

toot(login_user="user0001", status="Hello World, from user0001!")

2024-07-06T15:47:11.593388+0000 INFO user0001 successfully tooted the status.


## Post a status with additional settings

Note that `toot` is a synonym for `post_status`, but only takes the status text as input.

In [10]:
from mastodon_sim.mastodon_ops import post_status

post_status(
    login_user="user0002",
    status="Hello World, from user0002! This is an unlisted toot with a content warning.",
    visibility="unlisted",
    sensitive=True,
    spoiler_text="Content warning",
)

2024-07-06T15:47:12.106037+0000 INFO user0002 successfully posted the status.


## Send a DM

Send a DM from `user0001` to `user0002`.

In [11]:
post_status(
    login_user="user0001",
    status="@user0002 Hello, this is a direct message!",
    visibility="direct",
)

2024-07-06T15:47:12.588918+0000 INFO user0001 successfully posted the status.


###

## Post with media attachments

In [12]:
image_file = "../infrastructure/mastodon-on-aws/architecture.png"

post_status(
    login_user="user0001",
    status="Check out this image!",
    media_files=[image_file],
)

2024-07-06T15:47:15.134832+0000 INFO user0001 successfully posted the status.


## Create a poll

In [13]:
post_status(
    login_user="user0001",
    status="What's your favorite color?",
    poll_options=["Red", "Blue", "Green"],
    poll_expires_in=86400,
)

2024-07-06T15:47:15.577274+0000 INFO user0001 successfully posted the status.


## Get the public timeline

Get the entire pubic timeline in reverse chronological order, up to `limit`.

In [14]:
from mastodon_sim.mastodon_ops import get_public_timeline, print_timeline

timeline = get_public_timeline(limit=None)
print_timeline(timeline)

2024-07-06T15:47:15.773697+0000 INFO Retrieved 3 posts from the public timeline.
----------------------------------------
Post ID: 112740324837221417
Created At: 2024-07-06 15:47:15.529000+00:00
User: Bob (@user0001)
Content: <p>What&#39;s your favorite color?</p>
URL: https://social-sandbox.com/@user0001/112740324837221417
Favourites: 0, Reblogs: 0
----------------------------------------
Post ID: 112740324807978802
Created At: 2024-07-06 15:47:15.083000+00:00
User: Bob (@user0001)
Content: <p>Check out this image!</p>
URL: https://social-sandbox.com/@user0001/112740324807978802
Favourites: 0, Reblogs: 0
----------------------------------------
Post ID: 112740324575393974
Created At: 2024-07-06 15:47:11.536000+00:00
User: Bob (@user0001)
Content: <p>Hello World, from user0001!</p>
URL: https://social-sandbox.com/@user0001/112740324575393974
Favourites: 0, Reblogs: 0
----------------------------------------


## Get own timeline

Retrieve `user0002`'s timeline.

In [15]:
from mastodon_sim.mastodon_ops import get_own_timeline

timeline = get_own_timeline("user0002")
print_timeline(timeline)

2024-07-06T15:47:16.148837+0000 INFO user0002 retrieved 1 posts from their own timeline.
----------------------------------------
Post ID: 112740324609888646
Created At: 2024-07-06 15:47:12.062000+00:00
User:  (@user0002)
Content: <p>Hello World, from user0002! This is an unlisted toot with a content warning.</p>
URL: https://social-sandbox.com/@user0002/112740324609888646
Favourites: 0, Reblogs: 0
----------------------------------------


## Read a user's timeline (non-global)

From the perspective of `user0002`, read `user0001`'s timeline.

**Accessible Information**:
  - `user0002` can read `user0001`'s public and unlisted posts, as well as follower-only posts if `user0002` is a follower.
  - Details available include post ID, creation date, username, display name, content, URL, favourites count, and reblogs count.

**Restricted Information**:
  - `user0002` cannot access private posts or direct messages.

**Impact of Being Blocked**:
  - If `user0001` has blocked `user0002`, `user0002` cannot retrieve any posts from `user0001`'s timeline.
  - The function will return an empty list due to the block enforced by the Mastodon API.

In [16]:
from mastodon_sim.mastodon_ops import get_user_timeline

timeline = get_user_timeline(login_user="user0002", target_user="user0001", limit=None)
print_timeline(timeline)

2024-07-06T15:47:16.695888+0000 INFO user0002 retrieved 4 posts from user0001's timeline.
----------------------------------------
Post ID: 112740324837221417
Created At: 2024-07-06 15:47:15.529000+00:00
User: Bob (@user0001)
Content: <p>What&#39;s your favorite color?</p>
URL: https://social-sandbox.com/@user0001/112740324837221417
Favourites: 0, Reblogs: 0
----------------------------------------
Post ID: 112740324807978802
Created At: 2024-07-06 15:47:15.083000+00:00
User: Bob (@user0001)
Content: <p>Check out this image!</p>
URL: https://social-sandbox.com/@user0001/112740324807978802
Favourites: 0, Reblogs: 0
----------------------------------------
Post ID: 112740324642218668
Created At: 2024-07-06 15:47:12.555000+00:00
User: Bob (@user0001)
Content: <p><span class="h-card" translate="no"><a href="https://social-sandbox.com/@user0002" class="u-url mention">@<span>user0002</span></a></span> Hello, this is a direct message!</p>
URL: https://social-sandbox.com/@user0001/112740324642

## Block / unblock a user

Make `user0001` block or unblock `user0002`.

In [17]:
from mastodon_sim.mastodon_ops import block_user, unblock_user

# user001 blocks user002
block_user(login_user="user0001", target_user="user0002")

# user002 tries to read user001's timeline, but retrieves zero toots
timeline = get_user_timeline(login_user="user0002", target_user="user0001")

# user001 unblocks user002
unblock_user(login_user="user0001", target_user="user0002")

# user002 tries to read user001's timeline again, and retrieves the toots
timeline = get_user_timeline(login_user="user0002", target_user="user0001")

2024-07-06T15:47:19.039248+0000 INFO user0001 has blocked user0002.
2024-07-06T15:47:19.443103+0000 INFO user0002 retrieved 0 posts from user0001's timeline.
2024-07-06T15:47:19.911564+0000 INFO user0001 has unblocked user0002.
2024-07-06T15:47:20.425491+0000 INFO user0002 retrieved 4 posts from user0001's timeline.


## Like (favorite) a toot

Make `user0002` like `user0001`'s toot with ID `toot_id`.

In [18]:
from mastodon_sim.mastodon_ops import like_toot

timeline = get_user_timeline(login_user="user0002", target_user="user0001")

# Most recent toot is index 0 (reverse chronological order)
most_recent_toot_id = timeline[0]["id"]

like_toot(login_user="user0002", target_user="user0001", toot_id=most_recent_toot_id)

2024-07-06T15:47:20.960751+0000 INFO user0002 retrieved 4 posts from user0001's timeline.
2024-07-06T15:47:21.447589+0000 INFO user0002 liked post 112740324837221417 from user0001.


## Boost (reblog) a toot

Make `user0002` boost `user0001`'s toot with ID `toot_id`.

In [19]:
from mastodon_sim.mastodon_ops import boost_toot

timeline = get_user_timeline(login_user="user0002", target_user="user0001")
most_recent_toot_id = timeline[0]["id"]

boost_toot(login_user="user0002", target_user="user0001", toot_id=most_recent_toot_id)

2024-07-06T15:47:21.904489+0000 INFO user0002 retrieved 4 posts from user0001's timeline.
2024-07-06T15:47:22.304342+0000 INFO user0002 boosted post 112740324837221417 from user0001.


## Post a reply

Make `user0002` reply to `user0001`'s post (a poll).

In [20]:
timeline = get_user_timeline(login_user="user0002", target_user="user0001")
print(f"Last toot: {timeline[0]['content']}")
most_recent_toot_id = timeline[0]["id"]

post_status(
    login_user="user0002",
    status="This is a great poll!",
    in_reply_to_id=most_recent_toot_id,
)

2024-07-06T15:47:22.773504+0000 INFO user0002 retrieved 4 posts from user0001's timeline.
Last toot: <p>What&#39;s your favorite color?</p>
2024-07-06T15:47:23.166927+0000 INFO user0002 successfully posted the status.


## Delete post(s)

Make `user0002` delete their last post (the reply to `user0001`'s poll).

In [21]:
from mastodon_sim.mastodon_ops import delete_posts

# Check user0002's number of toots
timeline = get_own_timeline("user0002")

delete_posts(
    login_user="user0002",
    recent_count=1,
)

# Check user0002's number of toots again, should be 1 less
timeline = get_own_timeline("user0002")

2024-07-06T15:47:23.556434+0000 INFO user0002 retrieved 3 posts from their own timeline.
2024-07-06T15:47:24.075616+0000 INFO Successfully deleted post with ID: 112740325335123874
2024-07-06T15:47:24.076892+0000 INFO Deletion process completed. Attempted to delete 1 post(s).
2024-07-06T15:47:24.571325+0000 INFO user0002 retrieved 2 posts from their own timeline.


Delete a post by post ID.

In [22]:
# Get user0002's first toot (and print the number of toots)
timeline = get_own_timeline("user0002")
first_toot_id = timeline[0]["id"]

delete_posts(
    login_user="user0002",
    post_ids=[first_toot_id],
)

# Check user0002's number of toots again, should be 1 less
timeline = get_own_timeline("user0002")

2024-07-06T15:47:24.966238+0000 INFO user0002 retrieved 2 posts from their own timeline.
2024-07-06T15:47:25.397850+0000 INFO Successfully deleted post with ID: 112740325276785237
2024-07-06T15:47:25.398752+0000 INFO Deletion process completed. Attempted to delete 1 post(s).
2024-07-06T15:47:25.813528+0000 INFO user0002 retrieved 1 posts from their own timeline.


## Delete all posts

Delete all posts for a list of users.

In [23]:
for user in ["user0001", "user0002"]:
    delete_posts(login_user=user, delete_all=True, skip_confirm=True)

2024-07-06T15:47:26.422476+0000 INFO Successfully deleted post with ID: 112740324837221417
2024-07-06T15:47:26.494774+0000 INFO Successfully deleted post with ID: 112740324807978802
2024-07-06T15:47:26.561984+0000 INFO Successfully deleted post with ID: 112740324642218668
2024-07-06T15:47:26.627087+0000 INFO Successfully deleted post with ID: 112740324575393974
2024-07-06T15:47:26.628684+0000 INFO Deletion process completed. Attempted to delete 4 post(s).
2024-07-06T15:47:27.204815+0000 INFO Successfully deleted post with ID: 112740324609888646
2024-07-06T15:47:27.206186+0000 INFO Deletion process completed. Attempted to delete 1 post(s).


## Reset users

Delete user's posts, likes, boosts, and reset display name and bio.

In [28]:
from mastodon_sim.mastodon_ops import reset_user

for user in ["user0001", "user0002"]:
    reset_user(user, skip_confirm=True)

assert not len(get_public_timeline(limit=None)), "Public timeline should be empty now"

2024-07-06T15:49:05.058296+0000 INFO Deleting all posts for user user0001...
2024-07-06T15:49:05.509978+0000 ERROR No posts specified for deletion.
2024-07-06T15:49:05.513055+0000 INFO Removing favourites and boosts for user user0001...
2024-07-06T15:49:05.566871+0000 INFO Removed 0 favourites.
2024-07-06T15:49:05.658897+0000 INFO Removed 0 boosts.
2024-07-06T15:49:05.662638+0000 INFO Resetting profile information for user user0001...
2024-07-06T15:49:05.721299+0000 INFO Profile information reset successfully.
2024-07-06T15:49:05.724322+0000 INFO Comprehensive reset process completed for user: user0001
2024-07-06T15:49:06.087357+0000 INFO Deleting all posts for user user0002...
2024-07-06T15:49:06.562995+0000 ERROR No posts specified for deletion.
2024-07-06T15:49:06.563972+0000 INFO Removing favourites and boosts for user user0002...
2024-07-06T15:49:06.632633+0000 INFO Removed 0 favourites.
2024-07-06T15:49:06.770964+0000 INFO Removed 0 boosts.
2024-07-06T15:49:06.773626+0000 INFO Re

And that's it so far!

## [TODO] Operations to add:

- notifications
- notifications_clear
- favourites
- bookmarks
- account_mute
- account_unmute
- timeline_hashtag
- conversations
- trending_tags
- admin_account_moderate
- status_update

https://mastodonpy.readthedocs.io/en/stable/index.html